In [12]:
import numpy as np
from generator import Croupier

Импортируем класс Croupier, который отвечает за генерацию данных и создадим экземпляр класса с заданными значениями вероятности для нечестного кубика, а так же вероятности перехода с честного кубика на нечестный и на себя самого, и вероятности перехода с нечестного кубика на себя и на честный. Проверим что получаем выпавшие знаечение кубика, а так же метку кубика

In [16]:
casino = Croupier([0.1,0.1,0.1,0.1,0.1,1/2],[0.05,0.95],[0.9,0.1])
for i in range(10):
    print(casino())

(1, 'F')
(3, 'F')
(3, 'U')
(1, 'U')
(4, 'U')
(5, 'F')
(3, 'F')
(3, 'F')
(1, 'F')
(2, 'F')


Сгенерируем датасеты размером 1 000, 10 000 и 100 000

In [17]:
dataset300 = []
dataset300Labels = []

for i in range(300):
    data, label = casino()
    dataset300.append(data)
    dataset300Labels.append(label)

In [98]:
dataset1000 = []
dataset1000Labels = []

for i in range(1000):
    data, label = casino()
    dataset1000.append(data)
    dataset1000Labels.append(label)

In [30]:
dataset10000 = []
dataset10000Labels = []

for i in range(10000):
    data, label = casino()
    dataset10000.append(data)
    dataset10000Labels.append(label)

In [31]:
dataset100000 = []
dataset100000Labels = []

for i in range(100000):
    data, label = casino()
    dataset100000.append(data)
    dataset100000Labels.append(label)

Определим базовый класс алгоритма

In [110]:
class baseAlgo():
    def __init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None):
        self.data = data
        self.L = len(data)
        
        if fairDice == None: self.fairDice = np.log([1/6 for i in range(6)])
        else: self.fairDice = np.log(fairDice)
        self.unfairDice = np.log(unfairDice)
         
        self.toFairState = np.log([fairDiceStateProbability[0],unfairDiceStateProbability[0]])
        self.toUnfariState = np.log([fairDiceStateProbability[1],unfairDiceStateProbability[1]])
        
        self.fairDiceStateProbability = np.log(fairDiceStateProbability)
        self.unfairDiceStateProbability = np.log(unfairDiceStateProbability)
        
        #if fairDice == None: self.fairDice = np.array([1/6 for i in range(6)])
        #else: self.fairDice = np.array(fairDice)
        #self.unfairDice = np.array(unfairDice)
         
        #self.toFairState = np.array([fairDiceStateProbability[0],unfairDiceStateProbability[0]])
        #self.toUnfariState = np.array([fairDiceStateProbability[1],unfairDiceStateProbability[1]])
        
        #self.fairDiceStateProbability = np.array(fairDiceStateProbability)
        #self.unfairDiceStateProbability = np.array(unfairDiceStateProbability)
        
        self.states = ['Fair','Unfair']
        self.dices = [self.fairDice,self.unfairDice]
        self.transitions = [self.toFairState, self.toUnfariState]

In [130]:
class algorithmViterbi(baseAlgo):
    def __init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None):
        baseAlgo.__init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None)
        
        self.v = None
        self.ptr = None
        self.path = None

    def step(self,index):
        self.v.append([])
        self.ptr.append([])
        previuosState = self.v[index-1]
        for transitionProbability, probability in zip(self.transitions, self.dices):
            observedProbability = probability[self.data[index]-1]
            #print("PROBABILITY IN CURRENT STATE {}".format(probability))
            #print("DATA {} :PROBABILITY IN CURRENT STATE {}".format(self.data[index],observedProbability))
            
            #probability = observedProbability * np.max(previuosState*transitionProbability)
            #ptr = 'F' if np.argmax(previuosState*transitionProbability) == 0 else 'U'
            
            #print("TRANSITION PROBABILITY: {}".format(transitionProbability))
            
            thisPrrobability = observedProbability + np.max(previuosState + transitionProbability)
            ptr = 'F' if np.argmax(previuosState + transitionProbability) == 0 else 'U'
            #print('PTR: {}'.format(previuosState + transitionProbability))
            
            self.v[-1].append(thisPrrobability)
            self.ptr[-1].append(ptr)
            
    def __call__(self):
        if self.path == None: self.initialization()
        return self.path
        
        
    def initialization(self):
        
        self.ptr = []
        self.v = []
        self.path = []
        
        self.v.append([1,0]) # Не уверен
        
        # Рекурсия
        for i in range(1,self.L):
            self.step(i)
            print('PTR, V: {} _ {}'.format(self.ptr[i-1], self.v[i]))
                
        # Завершение
        self.Probability = max(self.v[-1])
        ptr, index = ('F', 0) if np.argmax(self.v[-1]) == 0 else ('U', 1)
        #print("PTR,INDEX: {} _ {}".format(ptr, index))
        self.path.append(ptr)
        
        # Обратный ход
        for step in range(self.L - 2,0,-1):
            ptr, index = ('F', 0) if self.ptr[step][index] == 'F' else ('U', 1)
            #print(self.ptr[step])
            #print(self.v[step])
            #print("Step {} : PTR,INDEX: {} _ {}".format(step,ptr, index))
            self.path.append(ptr)
        self.path = self.path[::-1]
        
    def probability(self):
        return np.e**self.Probability

In [131]:
hackCasinoViterbi = algorithmViterbi(dataset300, [0.95,0.05],[0.1,0.9],[0.1,0.1,0.1,0.1,0.1,1/2])
result = hackCasinoViterbi()
hackCasinoViterbi.probability()

PTR, V: ['F', 'U'] _ [-0.8430527636156055, -2.407945608651872]
PTR, V: ['F', 'U'] _ [-2.686105527231211, -4.815891217303744]
PTR, V: ['F', 'U'] _ [-4.529158290846817, -7.223836825955615]
PTR, V: ['F', 'U'] _ [-6.372211054462422, -9.631782434607487]
PTR, V: ['F', 'F'] _ [-8.215263818078027, -11.670528421010458]
PTR, V: ['F', 'F'] _ [-10.058316581693632, -11.904143272191963]
PTR, V: ['F', 'U'] _ [-11.901369345309236, -14.312088880843834]
PTR, V: ['F', 'U'] _ [-13.74442210892484, -16.720034489495706]
PTR, V: ['F', 'F'] _ [-15.587474872540446, -19.042739475472878]
PTR, V: ['F', 'F'] _ [-17.43052763615605, -20.885792239088484]
PTR, V: ['F', 'F'] _ [-19.273580399771657, -21.119407090269984]
PTR, V: ['F', 'U'] _ [-21.116633163387263, -23.527352698921856]
PTR, V: ['F', 'U'] _ [-22.95968592700287, -24.325860395139625]
PTR, V: ['F', 'U'] _ [-24.802738690618476, -25.124368091357393]
PTR, V: ['F', 'U'] _ [-26.645791454234082, -25.922875787575162]
PTR, V: ['F', 'U'] _ [-28.48884421784969, -28.33082

1.2776991284738255e-239

In [119]:
result

['F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F'

In [26]:
class forwardAlgorithm(baseAlgo):
    def __init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None):
        baseAlgo.__init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None)
        
        self.f = np.array([])
        self.pathProbability = None
    
    def step(self, index):
        self.f.append([])
        for transitionProbability, probability in zip(self.transitions, self.dices):
            observedProbability = probability[self.data[index]]
            previuosState = self.f[index-1]
            """
            observedProbability: вероятсность наблюдения при текущем состояние
            transitionProbability: вероятность перехода с кубика на кубик
            """
            probability = observedProbability * sum(previuosState * transitionProbability)
            self.f[-1].append(probability)
    
    def __call__(self):
        if self.pathProbability == None: self.initialization()
        return self.pathProbability
    
    def initialization(self):
        # Init
        self.f.append([1,0]) # Не уверен
        # Recursion
        for i in range(1,self.L):
            self.step(i)
        self.pathProbability = sum(self.f[-1])

In [ ]:
hackCasino = 

In [27]:
class backwardAlgorithm(baseAlgo):
    def __init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None):
        baseAlgo.__init__(self, data, fairDiceStateProbability,
                 unfairDiceStateProbability, unfairDice, fairDice = None)
        
        self.b = np.array([])
        self.pathProbability = None
    
    def step(self, index):
        self.b.append([])
        for transitionProbability in self.transitions: # first fair then unfair
            observedProbability = np.array([])
            for probability in self.dices
                observedProbability.append(probability[self.data[index]])
            nextState = self.b[index-1]
            
            probability = sum(transitionProbability * observedProbability * nextState)
            self.b[-1].append(probability)
    
    def __call__(self):
        if self.pathProbability == None: self.initialization()
        return self.pathProbability
    
    def initialization(self):
        #Init self.L - 1 КАК ИНИЦИАОИЗИРОВАТЬ?
        pass
        #Recursion
        for i in range(2,self.L): # С двойки?
            self.step(i)
        # Finish # перевернуть b
        self.b = self.b[::-1]
        pass

In [11]:
a = np.array([,1/2])
np.log(a)

array([-0.69314718, -0.69314718])